In [1]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [87]:
def AddDummy(data, column, value): #making it as a separate function not as a method to be able to reuse it in different projects easier
    data[value] = data[column].str.contains(value).astype('int')
    return data

class CarData:
    
    missing = -1
    duplicates = -1
    
    price_outliers = -1
    mileage_outliers = -1
    year_outliers = -1
    total_discard = -1
    
    def __init__(self, path, price_outlier_mt = 200000, mileage_outlier_mt = 400000,
                 year_outlier_lt = 1995, engine_outlier_mt = 4000, engine_outlier_lt = 750,
                brands = ['ford']):
        
        #define outliers values
        self.price_outlier_mt = price_outlier_mt
        self.mileage_outlier_mt = mileage_outlier_mt
        self.year_outlier_lt =  year_outlier_lt
        self.engine_outlier_mt = engine_outlier_mt
        self.engine_outlier_lt = engine_outlier_lt
        self.brands = brands
        
        #defince variable data types
        self.numeric_variables = ['price', 'mileage_km', 'engine_cm3', 'year']
        self.categorical_variables = ['engine_type', 'city', 'province']
        
        #read all .csv files from the directory
        self.data = pd.concat(map(pd.read_csv, glob.glob(os.path.join(path, "*.csv"))), sort=False)
        
        #drop the duplicates and save the number of duplicates - many duplicates due to data gathering method
        self.duplicates = len(self.data) - len(self.data.drop_duplicates())
        self.data.drop_duplicates(inplace = True)
        
        #rename columns
        self.data.columns = ['title', 'price', 'sub_title', 'mileage_km', 'year', 'engine_cm3',
                'engine_type', 'city', 'province', 'negotiable']
        
        #drop NaNs and save the number of rows dropped to the missing varaible
        self.missing = self.data['engine_type'].isna().sum()
        self.data.dropna(subset = ['engine_type'], axis = 'index', inplace = True)
        
        self.missing = self.missing + self.data['city'].isna().sum()
        self.data.dropna(subset = ['city'], axis = 'index', inplace = True)
        
        self.missing = self.missing + self.data['engine_cm3'].isna().sum()
        self.data.dropna(subset = ['engine_cm3'], axis = 'index', inplace = True)
        
        #clean up the columns
        self.data['price'] = self.data['price'].apply(lambda x: x.replace(",", ""))
        self.data['price'] = self.data['price'].apply(lambda x: x.replace(" ", "")).astype('int')
        
        self.data['mileage_km'] = self.data['mileage_km'].apply(lambda x: x.replace("km", ""))
        self.data['mileage_km'] = self.data['mileage_km'].apply(lambda x: x.replace(" ", "")).astype('float')
        
        self.data['engine_cm3'] = self.data['engine_cm3'].astype('str')
        self.data['engine_cm3'] = self.data['engine_cm3'].apply(lambda x: x.replace('cm3', ''))
        self.data['engine_cm3'] = self.data['engine_cm3'].apply(lambda x: x.replace(' ','')).astype('int')
        
        self.data['province'] = self.data['province'].astype('str')
        self.data['province'] = self.data['province'].apply(lambda x: x.replace('(',''))
        self.data['province'] = self.data['province'].apply(lambda x: x.replace(')',''))
        
        self.data['sub_title'] = self.data['sub_title'].astype('str') #may change that in the future - possible info loss due to lowercase
        self.data['sub_title'] = self.data['sub_title'].apply(lambda x: x.lower())
        
        self.data['title'] = self.data['title'].astype('str') #may change that in the future - possible info loss due to lowercase
        self.data['title'] = self.data['title'].apply(lambda x: x.lower())
    
        #Add ID column
        self.data.insert(loc = 0, column = 'ID', value = range(1, len(self.data)+1))

        #discard outliers and calculate the numbers
        self.total_discard = len(self.data) - len(self.data[(self.data['price'] <= self.price_outlier_mt) &
                                                        (self.data['mileage_km'] <= self.mileage_outlier_mt) &
                                                        (self.data['year'] >= self.year_outlier_lt) &
                                                        (self.data['engine_cm3'] <= self.engine_outlier_mt) &
                                                        (self.data['engine_cm3'] >= self.engine_outlier_lt)])        
        
        self.price_outliers = len(self.data[self.data['price'] > price_outlier_mt])
        self.data = self.data[self.data['price'] <= price_outlier_mt]
        
        self.mileage_outliers = len(self.data[self.data['mileage_km'] > mileage_outlier_mt])
        self.data = self.data[self.data['mileage_km'] <= mileage_outlier_mt]
        
        self.year_outliers = len(self.data[self.data['year'] < year_outlier_lt])
        self.data = self.data[self.data['year'] >= year_outlier_lt]
        
        self.engine_outliers = len(self.data[(self.data['engine_cm3'] > engine_outlier_mt) |
                                            (self.data['engine_cm3'] < engine_outlier_lt)])
        self.data = self.data[(self.data['engine_cm3'] <= engine_outlier_mt) & 
                             (self.data['engine_cm3'] >= engine_outlier_lt)]
        
    def describe(self):
        desc_stats = round(pd.DataFrame(
                        data = self.data[['price', 'mileage_km', 'year', 'engine_cm3']].describe(),
                        columns = self.data[['price', 'mileage_km', 'year', 'engine_cm3']].columns),2)
        return desc_stats
    
    def outliers(self):
        print('Offers with price greater than '+str(self.price_outlier_mt)+' have been discarded')
        print('The number of such offers = '+str(self.price_outliers))
        print('')
        print('Offers with mileage greater than '+str(self.mileage_outlier_mt)+' have been discarded')
        print('The number of such offers = '+str(self.mileage_outliers))
        print('')
        print('Offers with year lower than '+str(self.year_outlier_lt)+' have been discarded')
        print('The number of such offers = '+str(self.year_outliers))
        print('')
        print('Offers with engine_cm3 greater than '+str(self.engine_outlier_lt)+
              ' and lower than '+str(self.engine_outlier_mt)+' have been discarded')
        print('The number of such offers = '+str(self.engine_outliers))
        print('')
        print('Total number of discarded offers = '+str(self.total_discard)
              +'('+str(round(self.total_discard/len(self.data)*100,2))+'%)'
              +' - may be different to the sum of above due to overlap')
        
    def scatter_nox(self, var = 'all', figsize_1 = 7, figsize_2 = 5):
        if var == 'all':
            plt.rcParams["figure.figsize"] = (figsize_1,figsize_2)
            plt.scatter(y = self.data['mileage_km'], x = range(1, len(self.data)+1), s=1)
            plt.title('mileage_km')
            plt.show()

            plt.scatter(y = self.data['price'], x = range(1, len(self.data)+1), s=1)
            plt.title('price')
            plt.show()

            plt.scatter(y = self.data['year'], x = range(1, len(self.data)+1), s=1)
            plt.title('year')
            plt.show()

            plt.scatter(y = self.data['engine_cm3'], x = range(1, len(self.data)+1), s=1)
            plt.title('engine_cm3')
            plt.show()
        else:
            plt.rcParams["figure.figsize"] = (figsize_1,figsize_2)
            plt.scatter(y = self.data[var], x = range(1, len(self.data)+1), s=1)
            plt.title(var)
            plt.show()
            
    def scatter(self, var = 'all'):
        list_comb = []
        if var == 'all':
            for variable1 in enumerate(self.numeric_variables):
                for variable2 in enumerate(self.numeric_variables):
                    if variable1 != variable2 and variable1[1]+variable2[1] not in list_comb and variable2[1]+variable1[1] not in list_comb:
                        plt.scatter(y = self.data[variable1[1]], x = self.data[variable2[1]], s=1)
                        plt.title("Correlation between "+variable1[1]+' and '+variable2[1])
                        plt.ylabel(variable1[1])
                        plt.xlabel(variable2[1])
                        plt.show()
                        list_comb.append(variable1[1]+variable2[1])
        #else: - TO DO
    
    def hist(self, var = 'all', bins = 50):
        #prints histograms for numeric variables
        if var == 'all':
            for variable in enumerate(self.numeric_variables):
                plt.hist(x = self.data[variable[1]], bins = bins)
                plt.title(variable[1])
                plt.show()
     
    def price_cat_vars(self):
        for variable in enumerate(self.categorical_variables):
            # shows desrptive statistics of categorical variables
            print(x.data.groupby(x.data[variable[1]])['price'].describe())
            #the variables need further preprocessing
            
    def add_dummies(self, cat_list, columns):
        #adds dummmies from cat_list, checks in every column of columns
        for column in enumerate(columns):
            for cat in enumerate(cat_list):
                col_name = column[1] + '_' + cat[1]
                self.data[col_name] = self.data[column[1]].str.contains(cat[1]).astype('int')
                
                #append newly craeted varaibles to categorical variables
                if self.data[col_name].sum() > 0:
                    self.categorical_variables.append(col_name)
        return self
   

In [88]:
x = CarData('data/')

In [89]:
x = x.add_dummies(['ford'], ['title'])

In [91]:
x.data.head()

,ID,title,price,sub_title,mileage_km,year,engine_cm3,engine_type,city,province,negotiable,title_ford
0,1,smart fortwo cabrio,18800,cabrio automat nawigacja kamera led szwajcaria,37000.0,2016,999,Benzyna,Bielsko-Biała,Śląskie,"Do negocjacji, Faktura VAT",0
1,2,ford kuga,34700,nawigacja/kamera cofania/zadbany/,198000.0,2011,2000,Diesel,Dąbrowa,Pomorskie,Do negocjacji,1
2,3,chevrolet captiva 2.4,27900,2.4 136km klima alus navi dvd skóra 4x4 poleca...,246000.0,2009,2405,Benzyna,Płock,Mazowieckie,Do negocjacji,0
3,4,renault clio 1.2,37900,1.2 benz 73 km !! idealny !! full !! warszawa,15000.0,2017,1149,Benzyna,Warszawa,Mazowieckie,Do negocjacji,0
4,5,opel meriva,7900,mały przebieg 109 tyś/km bezwypadkowy,109000.0,2006,1400,Benzyna,Żyrardów,Mazowieckie,NaN,0


In [48]:
data = x.data

In [58]:
data2 = AddDummy(data, 'title', 'ford')

In [66]:
data['price'].sum()

786919459

In [3]:
x = CarData('data/')

In [4]:
x.outliers()

Offers with price greater than 200000 have been discarded
The number of such offers = 294

Offers with mileage greater than 400000 have been discarded
The number of such offers = 166

Offers with year lower than 1995 have been discarded
The number of such offers = 288

Offers with engine_cm3 greater than 750 and lower than 4000 have been discarded
The number of such offers = 378

Total number of discarded offers = 1126(4.68%) - may be different to the sum of above due to overlap


In [5]:
x.data.head()

,ID,title,price,sub_title,mileage_km,year,engine_cm3,engine_type,city,province,negotiable
0,1,Smart Fortwo cabrio,18800,Cabrio Automat Nawigacja Kamera Led Szwajcaria,37000.0,2016,999,Benzyna,Bielsko-Biała,Śląskie,"Do negocjacji, Faktura VAT"
1,2,Ford Kuga,34700,Nawigacja/kamera cofania/Zadbany/,198000.0,2011,2000,Diesel,Dąbrowa,Pomorskie,Do negocjacji
2,3,Chevrolet Captiva 2.4,27900,2.4 136km klima alus navi DVD skóra 4x4 Poleca...,246000.0,2009,2405,Benzyna,Płock,Mazowieckie,Do negocjacji
3,4,Renault Clio 1.2,37900,1.2 BENZ 73 KM !! IDEALNY !! FULL !! Warszawa,15000.0,2017,1149,Benzyna,Warszawa,Mazowieckie,Do negocjacji
4,5,Opel Meriva,7900,mały przebieg 109 tyś/km bezwypadkowy,109000.0,2006,1400,Benzyna,Żyrardów,Mazowieckie,NaN


In [20]:
x.missing, x.duplicates

(737, 8782)

In [64]:
x.describe()

,price,mileage_km,year,engine_cm3
count,24073.00,24073.00,24073.00,24073.00
mean,32688.88,159511.92,2009.88,1789.87
std,31457.86,83724.08,5.33,483.63
min,250.00,0.00,1995.00,796.00
25%,12200.00,99000.00,2006.00,1461.00
50%,22900.00,163000.00,2010.00,1794.00
75%,41800.00,217000.00,2014.00,1997.00
max,200000.00,400000.00,2020.00,4000.00


In [ ]:
x.scatter_nox()

In [ ]:
plt.hist(x.data['mileage_km'], bins = 100)
plt.title('abc')
plt.show()

plt.hist(x.data['price'],bins = 100)
plt.show()

plt.hist(x.data['year'], bins = 100)
plt.show()

plt.hist(x.data['engine_cm3'], bins = 100)
plt.show()

In [ ]:
x.data[x.data['price']>1100000]

In [ ]:
x.data[x.data['mileage_km']>1500000]

In [ ]:
x.data[x.data['year']<1990]

In [ ]:
round(pd.DataFrame(
    data = x.data[['price', 'mileage_km', 'year', 'engine_cm3']].describe(),
    columns = x.data[['price', 'mileage_km', 'year', 'engine_cm3']].columns),2)

In [ ]:
x.data.isnull().sum()

In [ ]:
data = pd.read_csv('run_results_21-04-2020.csv')

In [ ]:
data = ReadAndPreprocess(df)

In [ ]:
df = pd.read_csv('data/run_results_21-04-2020.csv')
df.head()

In [ ]:
df2 = pd.read_csv('data/run_results_25-04_2020.csv')
df2.head()

In [ ]:
df3 = pd.read_csv('data/run_results_26-04-2020.csv')
df3.head()

In [ ]:
x.df_head()